In [13]:
import re
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer

In [14]:
rootPath = "/"
relTrainCSVpath = "CONDA/data/CONDA_train.csv"
relTestCSVpath = "CONDA/data/CONDA_test.csv"
relValidationCSVpath = "CONDA/data/CONDA_valid.csv"

In [18]:
trainDF = pd.read_csv(os.path.join(relTrainCSVpath))
testDF = pd.read_csv(os.path.join(relTestCSVpath))
valDF = pd.read_csv(os.path.join(relValidationCSVpath))

In [19]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [20]:
texts = trainDF['utterance']
labels = trainDF['intentClass']
d = {'texts':texts, 'labels': labels}
train_DF = pd.DataFrame(d)
train_DF = train_DF.dropna()
texts = train_DF["texts"].tolist()

In [21]:
# trainDF2 = trainDF['utterance'].map(lambda x: tokenizer(str(x), truncation=True))

#trainDF['utterance'] = trainDF['utterance'].map(lambda x: tokenizer(str(x), truncation=True))
#train_Str = trainDF.utterance.values.tolist()
train_encodings = tokenizer(texts, truncation=True, padding=True)

In [22]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
import evaluate

accuracy = evaluate.load("accuracy")

In [24]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [25]:
# print(trainDF['intentClass'][:20])
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# trainDF['intentClassLabel'] = le.fit_transform(trainDF['intentClass'])
# trainDF[:10]
train_DF["labels"].tail()

26916    O
26917    O
26918    O
26919    O
26920    O
Name: labels, dtype: object

In [26]:
id2label = {0: "A", 1: "E", 2: "I", 3: "O"}
label2id = {"A": 0, "E": 1, "I": 2, "O": 3}

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [31]:
texts = valDF['utterance']
labels = valDF['intentClass']
d = {'texts':texts, 'labels': labels}
test_DF = pd.DataFrame(d)
test_DF = test_DF.dropna()

In [51]:
from torch.utils.data import Dataset
import torch

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        # texts = dataframe.texts.tolist()
        # self.texts = tokenizer(texts,truncation=True, padding=True)
        self.texts = dataframe.texts
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        # texts = str(self.texts[index])
        # texts = " ".join(texts.split())

        # # inputs = self.tokenizer.encode_plus(
        # #     texts,
        # #     None,
        # #     add_special_tokens=True,
        # #     max_length=self.max_len,
        # #     pad_to_max_length=True,
        # #     return_token_type_ids=True
        # # )
        # # ids = inputs['input_ids']
        # mask = inputs['attention_mask']
        # token_type_ids = inputs["token_type_ids"]


        # return {
        #     'ids': torch.tensor(ids, dtype=torch.long),
        #     'mask': torch.tensor(mask, dtype=torch.long),
        #     'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        #     'targets': torch.tensor(self.targets[index], dtype=torch.float)
        #     }
        item = {key: torch.tensor(val[index]) for key, val in self.texts.items()}
        item['labels'] = torch.tensor(self.labels[index])
        return item

In [52]:
trainDataset = CustomDataset(train_DF,tokenizer,max_len=32)
testDataset = CustomDataset(test_DF,tokenizer,max_len=32)

  0%|          | 0/3366 [11:16<?, ?it/s]


In [66]:
trainDataset[1]

c:\Users\Karan\Documents\AI_COURSE\aasd4011-2023-spring\venv\pytorch\lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


TypeError: new(): invalid data type 'str'

In [54]:
training_args = TrainingArguments(
    output_dir="temp_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainDataset,
    eval_dataset=testDataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

TypeError: new(): invalid data type 'str'